## Loading preprosed data

In [1]:
import pandas as pd

In [1]:
PATH = "saved_models/attention/version_1/version.cpkt"

In [3]:
file_path = "regression.csv"

In [4]:
data = pd.read_csv(file_path, sep=',', header=(0), skipinitialspace=True)

In [5]:
data['Y'].max(), data['Y'].min()

(1.8588609431999998, -1.9754904544999998)

In [6]:
norm_data = data.copy()
#norm_data['Y']=(norm_data['Y']-norm_data['Y'].min())/(norm_data['Y'].max()-norm_data['Y'].min())
norm_data['Y']=(norm_data['Y']-norm_data['Y'].mean())/norm_data['Y'].std()

In [7]:
norm_data['Y'].max(), norm_data['Y'].min()

(6.150541397957692, -6.28618553626624)

In [8]:
norm_data.iloc[:,:-1] = norm_data.iloc[:,:-1].astype(int)

In [9]:
from sklearn.model_selection import train_test_split

In [10]:
train, test = train_test_split(norm_data, test_size=0.2)

In [11]:
train.shape

(51260, 21)

In [12]:
train_data = train.iloc[:,:-1].values
train_label = train.iloc[:,-1].values
val_data = test.iloc[:,:-1].values
val_label = test.iloc[:,-1].values

In [13]:
train_data.shape, train_label.shape, val_data.shape, val_label.shape

((51260, 20), (51260,), (12816, 20), (12816,))

In [14]:
train_data[0], train_label[0]

(array([2, 2, 1, 0, 2, 3, 1, 2, 2, 3, 0, 1, 0, 0, 2, 1, 2, 1, 3, 1]),
 -2.7490868759706966)

In [15]:
import numpy as np
NUM_OF_ACIDS = 4
EMBEDDING_SIZE = 32
NUM_CLASSES = np.amax(val_label, axis=0)+1

In [16]:
NUM_EPOCH=5

## Model

### One hot encoding

In [17]:
import tensorflow as tf
tf.__version__

'1.5.0'

# Training

In [18]:
def train_input():
    return (tf.data.Dataset.from_tensor_slices((train_data, train_label))
            .shuffle(buffer_size=10000, reshuffle_each_iteration=True)
            .batch(64)
            .repeat(NUM_EPOCH))

In [19]:
def eval_input():
    return (tf.data.Dataset.from_tensor_slices((val_data, val_label))
            .batch(64).repeat(1))

In [20]:
def model(features, is_training):
    acid_embeddings = tf.get_variable("acid_embeddings", [NUM_OF_ACIDS, EMBEDDING_SIZE])
    embedded_acids = tf.nn.embedding_lookup(acid_embeddings, features)
    embedded_acids_flatten = tf.layers.flatten(embedded_acids)
   # Build RNN cell
    encoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units)

    # Run Dynamic RNN
    #   encoder_outputs: [max_time, batch_size, num_units]
    #   encoder_state: [batch_size, num_units]
    encoder_outputs, encoder_state = tf.nn.dynamic_rnn(encoder_cell, encoder_emb_inp,
                                                       sequence_length=source_sequence_length, time_major=True)
    
    # Build RNN cell
    decoder_cell = tf.nn.rnn_cell.BasicLSTMCell(num_units)

    # Helper
    helper = tf.contrib.seq2seq.TrainingHelper(decoder_emb_inp, decoder_lengths, time_major=True)
    # Decoder
    decoder = tf.contrib.seq2seq.BasicDecoder(decoder_cell, helper, encoder_state,
                                              output_layer=projection_layer)
    # Dynamic decoding
    outputs, _ = tf.contrib.seq2seq.dynamic_decode(decoder, ...)
    logits = outputs.rnn_output

    return x

In [21]:
def model_fn(features, labels, mode, params):
    """The model_fn argument for creating an Estimator."""
    if mode == tf.estimator.ModeKeys.PREDICT:
        logits = model(features, is_training=False)
        return tf.estimator.EstimatorSpec(
            mode=tf.estimator.ModeKeys.PREDICT,
            predictions=logits)
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.AdamOptimizer(learning_rate=1e-4)
        logits = model(features, is_training=True)
        loss = tf.losses.absolute_difference(labels=labels, predictions=logits)
        return tf.estimator.EstimatorSpec(
            mode=tf.estimator.ModeKeys.TRAIN,
            loss=loss,
            train_op=optimizer.minimize(loss, tf.train.get_or_create_global_step()))
    if mode == tf.estimator.ModeKeys.EVAL:
        logits = model(features, is_training=False)
        loss = tf.losses.absolute_difference(labels=labels, predictions=logits)
        return tf.estimator.EstimatorSpec(
            mode=tf.estimator.ModeKeys.EVAL,
            loss=loss)


In [22]:
enzyme_classifier = tf.estimator.Estimator(
      model_fn=model_fn,
      model_dir=PATH)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'saved_models/regression/version_1/version.cpkt', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001AA6A3F77B8>, '_task_type': 'worker', '_task_id': 0, '_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [26]:
enzyme_classifier.train(input_fn=train_input) 

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Restoring parameters from saved_models/regression/version_1/version.cpkt\model.ckpt-8010
INFO:tensorflow:Saving checkpoints for 8011 into saved_models/regression/version_1/version.cpkt\model.ckpt.
INFO:tensorflow:loss = 0.6924429, step = 8011
INFO:tensorflow:global_step/sec: 79.1329
INFO:tensorflow:loss = 0.737671, step = 8111 (1.264 sec)
INFO:tensorflow:global_step/sec: 77.7774
INFO:tensorflow:loss = 0.5827149, step = 8211 (1.286 sec)
INFO:tensorflow:global_step/sec: 76.7883
INFO:tensorflow:loss = 0.654294, step = 8311 (1.302 sec)
INFO:tensorflow:global_step/sec: 94.7258
INFO:tensorflow:loss = 0.7169365, step = 8411 (1.056 sec)
INFO:tensorflow:global_step/sec: 79.0494
INFO:tensorflow:loss = 0.883518, step = 8511 (1.265 sec)
INFO:tensorflow:global_step/sec: 84.3069
INFO:tensorflow:loss = 0.7001387, step = 8611 (1.202 sec)
INFO:tensorflow:global_step/sec: 76.856
INFO:tensorflow:loss = 0.8358129, step = 8711 (1.286 sec)
INFO:ten

In [27]:
eval_results = enzyme_classifier.evaluate(input_fn=eval_input)
print()
print('Evaluation results:\n\t%s' % eval_results)

INFO:tensorflow:Starting evaluation at 2018-03-07-19:48:24
INFO:tensorflow:Restoring parameters from saved_models/regression/version_1/version.cpkt\model.ckpt-12015
INFO:tensorflow:Finished evaluation at 2018-03-07-19:48:26
INFO:tensorflow:Saving dict for global step 12015: global_step = 12015, loss = 0.791159

Evaluation results:
	{'loss': 0.791159, 'global_step': 12015}


In [ ]:
def prod_input():
    index = 10
    print (val_data[0:index])
    print (val_label[0:index])
    return (tf.data.Dataset.from_tensor_slices((val_data[0:index], val_label[0:index])).batch(index))

In [26]:
predict = enzyme_classifier.predict(input_fn=prod_input)
print (predict)
for p in predict:
    print(p)

<generator object Estimator.predict at 0x000002A493B3A6D0>
[[2 2 2 3 0 0 0 2 1 1 1 0 0 1 2 2 2 1 0 0]
 [1 3 0 0 3 2 1 3 1 3 1 0 2 3 0 3 1 3 1 3]
 [1 2 0 2 3 0 3 2 1 3 2 1 1 0 1 3 1 1 1 1]
 [1 0 3 2 1 3 2 1 0 1 1 0 0 2 2 1 0 3 0 1]
 [1 2 1 0 1 1 3 2 2 1 1 2 2 3 0 2 3 0 2 3]
 [3 3 3 2 1 1 2 2 3 0 2 1 1 1 2 2 0 2 0 1]
 [1 3 0 1 0 3 1 2 3 1 3 2 3 2 1 2 1 1 1 0]
 [1 2 2 3 1 2 2 1 3 2 1 1 2 3 2 3 1 2 1 3]
 [0 3 3 2 2 2 0 1 1 2 3 1 1 1 2 3 2 1 3 0]
 [3 2 0 2 1 0 2 2 3 1 2 1 1 1 0 2 3 1 3 1]]
[ 1.8371417  -0.76188853 -1.00000062 -0.99341339 -1.06843164  1.06686331
 -0.74164628  0.72892113  0.36989805  0.81223369]
INFO:tensorflow:Restoring parameters from saved_models/regression/version_1/version.cpkt\model.ckpt-4005
0.22486427
0.11091128
-0.090491116
0.014593732
0.048556305
-0.07160731
0.0857504
-0.035996374
-0.06441446
-0.27501512
